In [1]:
import onnx 
from onnx import shape_inference
import onnxruntime  as rt
import pickle
import torch
from ten_vad_reconstructed import TenVAD

In [2]:
model =  onnx.load("ten_vad_exported.onnx")
model = shape_inference.infer_shapes(model)
state_dict = torch.load("ten_vad_onnx_weights.pth")
torch_model = TenVAD().to('cuda')
torch_model.load_state_dict(state_dict)
torch_model.eval()


TenVAD(
  (conv_dw): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv_pw): Conv2d(1, 16, kernel_size=(1, 1), stride=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=(1, 3), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (sep1_dw): Conv2d(16, 16, kernel_size=(1, 3), stride=(2, 2), padding=(0, 1), groups=16, bias=False)
  (sep1_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
  (sep2_dw): Conv2d(16, 16, kernel_size=(1, 3), stride=(2, 2), groups=16, bias=False)
  (sep2_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
  (lstm1): LSTM(80, 64, batch_first=True)
  (lstm2): LSTM(64, 64, batch_first=True)
  (fc1): Linear(in_features=128, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [3]:
new_outputs = []
for val_info in model.graph.value_info:
    new_outputs.append(val_info)
    
for o in new_outputs:
    model.graph.output.append(o)

onnx.save(model, "debug_torch_model.onnx")


In [4]:
orig_model = onnx.load("src/onnx_model/ten-vad.onnx")
orig_model =  onnx.load("src/onnx_model/ten-vad.onnx")
orig_model = shape_inference.infer_shapes(orig_model)
new_outputs = []
for val_info in orig_model.graph.value_info:
    new_outputs.append(val_info)
    
for o in new_outputs:
    orig_model.graph.output.append(o)

onnx.save(orig_model, "debug_onnx_model.onnx")


In [5]:
model_name = "debug_torch_model.onnx"
model = onnx.load(model_name)
with open("pickle_input.pkl", "rb") as f:
    input_t = pickle.load(f)

session = rt.InferenceSession(model_name, providers=['CPUExecutionProvider'])
output_names = [ o.name for o in session.get_outputs()]
raw_outputs = session.run(None, input_t)
outputs_dict = dict(zip(output_names, raw_outputs))
l = list(outputs_dict.keys())
for i in l:
    print(i)


output
159
160
220
221
/Unsqueeze_output_0
/Constant_output_0
/Reshape_output_0
/conv_dw/Conv_output_0
/conv_pw/Conv_output_0
/relu/Relu_output_0
/pool/MaxPool_output_0
/sep1_dw/Conv_output_0
/sep1_pw/Conv_output_0
/Constant_1_output_0
/Shape_output_0
/Gather_output_0
/Constant_2_output_0
/Equal_output_0
/If_output_0
/relu_1/Relu_output_0
/Unsqueeze_1_output_0
/Constant_3_output_0
/Constant_4_output_0
/ConstantOfShape_output_0
/Concat_output_0
/Constant_5_output_0
/Reshape_1_output_0
/Constant_6_output_0
/Constant_7_output_0
/Constant_8_output_0
/Constant_9_output_0
/Slice_output_0
/Transpose_output_0
/Constant_10_output_0
/Reshape_2_output_0
/Cast_output_0
/Constant_11_output_0
/Pad_output_0
/sep2_dw/Conv_output_0
/sep2_pw/Conv_output_0
/Constant_12_output_0
/Shape_1_output_0
/Gather_1_output_0
/Constant_13_output_0
/Equal_1_output_0
/If_1_output_0
/relu_2/Relu_output_0
/Transpose_1_output_0
/Shape_2_output_0
/Constant_14_output_0
/Gather_2_output_0
/Shape_3_output_0
/Constant_15_outp

In [6]:
model_name = "debug_onnx_model.onnx"
orig_model = onnx.load(model_name)
with open("pickle_input.pkl", "rb") as f:
    input_t = pickle.load(f)

session2 = rt.InferenceSession(model_name, providers=['CPUExecutionProvider'])
output_names2 = [ o.name for o in session2.get_outputs()]
raw_outputs2 = session2.run(None, input_t)
outputs_dict2 = dict(zip(output_names2, raw_outputs2))
l = list(outputs_dict2.keys())
for i in l:
    print(i)


output_1
output_2
output_3
output_6
output_7
Unsqueeze__97:0
Unsqueeze__95:0
Unsqueeze__68:0
Unsqueeze__66:0
StatefulPartitionedCall/vad_model/ExpandDims:0
StatefulPartitionedCall/vad_model/separable_conv2d/separable_conv2d/depthwise__111:0
StatefulPartitionedCall/vad_model/separable_conv2d/separable_conv2d/depthwise:0
StatefulPartitionedCall/vad_model/separable_conv2d/BiasAdd:0
StatefulPartitionedCall/vad_model/separable_conv2d/Relu:0
StatefulPartitionedCall/vad_model/separable_conv1d/separable_conv2d/depthwise__127:0
StatefulPartitionedCall/vad_model/separable_conv1d/separable_conv2d/depthwise:0
StatefulPartitionedCall/vad_model/separable_conv1d/BiasAdd:0
StatefulPartitionedCall/vad_model/separable_conv1d/Squeeze:0
StatefulPartitionedCall/vad_model/separable_conv1d/Relu:0
StatefulPartitionedCall/vad_model/separable_conv1d_1/ExpandDims:0
StatefulPartitionedCall/vad_model/separable_conv1d_1/separable_conv2d/depthwise:0
StatefulPartitionedCall/vad_model/separable_conv1d_1/BiasAdd:0
Stat

In [9]:
inputs = {k:torch.from_numpy(v).to("cuda").float() for k,v in input_t.items()}
with torch.inference_mode():
    outputs = torch_model(**inputs)

outputs.shape, outputs

(torch.Size([1, 1, 64]),
 tensor([[[ 1.1455e-01,  2.2319e-03,  3.4622e-05,  1.5677e-02,  5.0729e-02,
            2.6100e-03, -2.5539e-01, -1.1169e-02,  2.3027e-02,  2.3777e-01,
            4.1674e-02, -1.0629e-02,  2.0085e-02, -3.4918e-03,  6.1300e-03,
            3.7393e-04, -7.2495e-01, -9.4491e-04,  1.0627e-02,  5.1566e-03,
           -1.5549e-01,  2.6605e-02,  6.0866e-05,  7.1047e-02, -7.1336e-01,
           -1.1758e-02,  5.8289e-02,  4.9545e-03,  8.2858e-03, -3.7118e-01,
            6.7800e-01, -1.3712e-01,  2.7457e-03, -2.5531e-01,  2.0380e-01,
            1.0210e-03, -1.6683e-02,  1.6065e-02,  1.0140e-01, -7.1636e-01,
            4.0471e-03, -6.9450e-01, -9.5008e-03, -1.7269e-06,  6.2494e-01,
            6.4456e-02,  3.3084e-01,  7.3581e-02, -1.4559e-03, -2.4578e-01,
            6.3197e-02,  2.4946e-01,  4.2983e-02, -1.3051e-04, -1.9492e-01,
            9.6574e-05,  5.6413e-01,  2.3526e-04, -7.3422e-01, -3.4906e-01,
            3.7469e-02,  4.1834e-04,  1.2700e-03,  3.4934e-01]]

In [26]:
# onnx_output= outputs_dict2['StatefulPartitionedCall/vad_model/lstm/PartitionedCall/transpose:0']
# torch_output = outputs_dict['/Reshape_3_output_0']

onnx_output= outputs_dict2['LSTM__73:0']
# torch_output = outputs_dict['/Unsqueeze_4_output_0']

display(onnx_output.shape)#, display(torch_output.shape)
#fix onnx output 
onnx_output_fixed = onnx_output.squeeze(1)  # Remove num_directions dimension
onnx_output_fixed = onnx_output_fixed.transpose(1, 0, 2)
onnx_output_fixed





(1, 1, 1, 64)

array([[[-4.0561956e-01, -2.6092851e-03, -5.2212199e-05, -9.5241461e-03,
         -5.4733064e-03, -1.6409131e-02,  7.6739639e-03, -3.6653925e-03,
         -7.4510523e-03,  1.3249981e-01,  8.3374590e-02,  1.0331009e-03,
          3.5107995e-03, -3.5438277e-02,  6.4066141e-03,  2.9350645e-04,
          7.1306485e-01, -1.4669594e-03, -3.2577369e-02, -1.0248714e-02,
         -7.0155728e-01,  9.9166734e-03, -1.1119212e-03, -7.2317589e-03,
          7.5774437e-01, -2.4382159e-02,  7.2176002e-02, -4.7024649e-02,
         -7.5228459e-01,  7.8379503e-03,  7.2490573e-02, -7.1959072e-01,
         -1.2122970e-03, -4.6606508e-01, -7.9160623e-02,  7.1216526e-04,
          1.4785431e-02, -7.5878674e-04, -8.8518359e-02,  5.7699770e-01,
         -6.3017459e-04,  5.8952105e-01, -6.8233955e-01, -2.9505866e-02,
          7.1074140e-01, -7.9221632e-03, -8.5678108e-02, -1.9748281e-01,
         -3.2891501e-02, -6.7008495e-02,  1.9274218e-03, -5.1458341e-01,
         -6.2436926e-01, -6.7237146e-05, -5.8339804

In [24]:
onnx_output

array([[[[-4.0561956e-01, -2.6092851e-03, -5.2212199e-05,
          -9.5241461e-03, -5.4733064e-03, -1.6409131e-02,
           7.6739639e-03, -3.6653925e-03, -7.4510523e-03,
           1.3249981e-01,  8.3374590e-02,  1.0331009e-03,
           3.5107995e-03, -3.5438277e-02,  6.4066141e-03,
           2.9350645e-04,  7.1306485e-01, -1.4669594e-03,
          -3.2577369e-02, -1.0248714e-02, -7.0155728e-01,
           9.9166734e-03, -1.1119212e-03, -7.2317589e-03,
           7.5774437e-01, -2.4382159e-02,  7.2176002e-02,
          -4.7024649e-02, -7.5228459e-01,  7.8379503e-03,
           7.2490573e-02, -7.1959072e-01, -1.2122970e-03,
          -4.6606508e-01, -7.9160623e-02,  7.1216526e-04,
           1.4785431e-02, -7.5878674e-04, -8.8518359e-02,
           5.7699770e-01, -6.3017459e-04,  5.8952105e-01,
          -6.8233955e-01, -2.9505866e-02,  7.1074140e-01,
          -7.9221632e-03, -8.5678108e-02, -1.9748281e-01,
          -3.2891501e-02, -6.7008495e-02,  1.9274218e-03,
          -5.1

In [12]:
for i, node in enumerate(model.graph.node):
    print(i, node.name)

0 /Unsqueeze
1 /Constant
2 /Reshape
3 /conv_dw/Conv
4 /conv_pw/Conv
5 /relu/Relu
6 /pool/MaxPool
7 /sep1_dw/Conv
8 /sep1_pw/Conv
9 /Constant_1
10 /Shape
11 /Gather
12 /Constant_2
13 /Equal
14 /If
15 /relu_1/Relu
16 /Unsqueeze_1
17 /Constant_3
18 /Constant_4
19 /ConstantOfShape
20 /Concat
21 /Constant_5
22 /Reshape_1
23 /Constant_6
24 /Constant_7
25 /Constant_8
26 /Constant_9
27 /Slice
28 /Transpose
29 /Constant_10
30 /Reshape_2
31 /Cast
32 /Constant_11
33 /Pad
34 /sep2_dw/Conv
35 /sep2_pw/Conv
36 /Constant_12
37 /Shape_1
38 /Gather_1
39 /Constant_13
40 /Equal_1
41 /If_1
42 /relu_2/Relu
43 /Transpose_1
44 /Shape_2
45 /Constant_14
46 /Gather_2
47 /Shape_3
48 /Constant_15
49 /Gather_3
50 /Shape_4
51 /Constant_16
52 /Gather_4
53 /Mul
54 /Unsqueeze_2
55 /Constant_17
56 /Unsqueeze_3
57 /Concat_1
58 /Reshape_3
59 /Unsqueeze_4
60 /Unsqueeze_5
61 /Unsqueeze_6
62 /Unsqueeze_7
63 /lstm1/LSTM
64 /lstm1/Squeeze
65 /lstm2/LSTM
66 /lstm2/Squeeze
67 /Concat_2
68 /fc1/MatMul
69 /fc1/Add
70 /relu_3/Relu

In [38]:
model_lstm_node = model.graph.node[63]


In [14]:
for i, initializer in enumerate(model.graph.initializer):
    print(i, initializer.name)

0 conv_dw.weight
1 conv_pw.weight
2 conv_pw.bias
3 sep1_dw.weight
4 sep1_pw.weight
5 sep1_pw.bias
6 sep2_dw.weight
7 sep2_pw.weight
8 sep2_pw.bias
9 fc1.bias
10 fc2.bias
11 onnx::LSTM_255
12 onnx::LSTM_256
13 onnx::LSTM_257
14 onnx::LSTM_275
15 onnx::LSTM_276
16 onnx::LSTM_277
17 onnx::MatMul_278
18 onnx::MatMul_279


In [33]:
orig_lstm_node = orig_model.graph.node[21]

In [16]:
for i, initializer in enumerate(orig_model.graph.initializer):
    print(i, initializer.name)

0 new_shape__175
1 const_fold_opt__178
2 StatefulPartitionedCall/vad_model/separable_conv2d/separable_conv2d/ReadVariableOp_1:0
3 StatefulPartitionedCall/vad_model/separable_conv2d/BiasAdd/ReadVariableOp:0
4 const_fold_opt__179
5 StatefulPartitionedCall/vad_model/separable_conv1d/ExpandDims_2:0
6 StatefulPartitionedCall/vad_model/separable_conv1d/BiasAdd/ReadVariableOp:0
7 const_fold_opt__180
8 StatefulPartitionedCall/vad_model/separable_conv1d_1/ExpandDims_2:0
9 StatefulPartitionedCall/vad_model/separable_conv1d_1/BiasAdd/ReadVariableOp:0
10 new_shape__177
11 W0__70
12 R0__71
13 B0__72
14 new_shape__176
15 W0__99
16 R0__100
17 B0__101
18 StatefulPartitionedCall/vad_model/dense_3/Tensordot/free:0
19 StatefulPartitionedCall/vad_model/dense/Tensordot/Const_2:0
20 StatefulPartitionedCall/vad_model/dense/Tensordot/Reshape_shape__186
21 StatefulPartitionedCall/vad_model/dense_3/Tensordot/ReadVariableOp:0
22 StatefulPartitionedCall/vad_model/dense_3/BiasAdd/ReadVariableOp:0
23 StatefulPartit

In [17]:
W = model.graph.initializer[11]
R = model.graph.initializer[12]
B = model.graph.initializer[13]

In [12]:
W_ori = orig_model.graph.initializer[11]
R_ori = orig_model.graph.initializer[12]
B_ori = orig_model.graph.initializer[13]

In [13]:
W_torch = state_dict['lstm1.weight_ih_l0'].numpy()
R_torch = state_dict['lstm1.weight_hh_l0'].numpy()
B_torch = state_dict['lstm1.bias_ih_l0'].numpy()

In [18]:
W_ori.name = 'W'
R_ori.name = 'R'
B_ori.name = 'B'
import numpy as np
W_ori_weight = np.frombuffer(W_ori.raw_data, dtype=np.float32).reshape(W_ori.dims)
W_weight = np.frombuffer(W.raw_data, dtype=np.float32).reshape(W.dims)

R_ori_weight = np.frombuffer(R_ori.raw_data, dtype=np.float32).reshape(R_ori.dims)
R_weight = np.frombuffer(R.raw_data, dtype=np.float32).reshape(R.dims)

B_ori_weight = np.frombuffer(B_ori.raw_data, dtype=np.float32).reshape(B_ori.dims)
B_weight = np.frombuffer(B.raw_data, dtype=np.float32).reshape(B.dims)

W_ori_weight - W_weight, R_ori_weight - R_weight, B_ori_weight - B_weight

(array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [-0.1447528 , -0.2812652 , -0.9021763 , ...,  0.38451663,
           0.08557338, -0.23741567],
         [-0.33152345, -0.22362734, -0.00653614, ..., -0.6657253 ,
          -0.9935426 , -0.5834838 ],
         [-0.1252819 ,  0.27870315,  0.13105348, ...,  0.18257932,
           0.83363706,  0.32151163]]], shape=(1, 256, 80), dtype=float32),
 array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         ...

In [19]:
W_ori_weight

array([[[ 0.4867446 ,  0.47443914, -0.590404  , ..., -0.03301766,
          0.03501979, -0.10179292],
        [-0.4780938 , -0.2546657 , -0.42414826, ...,  0.63041824,
          0.26865023,  0.75948215],
        [-0.8182535 , -0.43482986, -0.21190313, ...,  0.07235646,
         -0.25456637, -0.1706717 ],
        ...,
        [ 0.06247028, -0.01019134, -0.66108805, ...,  0.32636583,
          0.27987358,  0.0863558 ],
        [ 0.03873489,  0.13783218,  0.075941  , ..., -0.39508176,
         -0.3009435 , -0.15503898],
        [-0.4833058 , -0.17500852, -0.10183387, ...,  0.33798772,
          0.694602  ,  0.58902663]]], shape=(1, 256, 80), dtype=float32)

In [22]:
#check lstm 
W_torch = state_dict['lstm1.weight_ih_l0'].numpy()
print(W_torch - W_ori_weight)

R_torch = state_dict['lstm1.weight_hh_l0'].numpy()
print(R_torch - R_ori_weight)

B_torch = state_dict['lstm1.bias_ih_l0'].numpy()
print(B_torch - B_ori_weight[:, :256])


[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0

In [70]:
W_torch_loaded = torch.load('ih_l0.pth')
W_torch_loaded.detach().numpy() - W_ori_weight

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], shape=(1, 256, 80), dtype=float32)

In [64]:
#check lstm 
W_torch = state_dict['lstm1.weight_ih_l0'].numpy()
print(W_torch - W_weight)

R_torch = state_dict['lstm1.weight_hh_l0'].numpy()
print(R_torch - R_weight)

[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [-0.1447528  -0.2812652  -0.9021763  ...  0.38451663  0.08557338
   -0.23741567]
  [-0.33152345 -0.22362734 -0.00653614 ... -0.6657253  -0.9935426
   -0.5834838 ]
  [-0.1252819   0.27870315  0.13105348 ...  0.18257932  0.83363706
    0.32151163]]]
[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.48578432  0.18361041  0.17614032 ...  0.28419352  0.21935251
    0.23873608]
  [ 0.9871485   0.25750008  0.3457416  ...  0.0128186   0.17374223
   -1.0447848 ]
  [ 0.6165874   0.00960901 -0.15370113 ... -0.12160549  0.13375849
   -0.1566813 ]]]


In [21]:
model.graph.initializer[13]

dims: 1
dims: 512
data_type: 1
name: "onnx::LSTM_257"
raw_data: "9\033\207\2766|\n\276\025>\325\276x\345\375\276\242-;\276E\230\325\276Yc$\275\r\245\204\276;\212\224\276\'\353\302\276\033\304~\275\226\361H\276\331WI\276\n*B\276\342\035I\277b\264\030\277\017\261G\276\337l\325\276t\347r\2762\t\327\276P\034\324=\204S\022\277\2475\372\275#b\346\276\315\311\336>U\016\214\276\326\304\372\276\235[\252\276\312\235\221>%bD\275\227\236\302\276v\334a=\334E8\276\246\215\375=\020~\240=\253\257\337\276\000\332\341=F\313\210=:\214\271\276\327\233\\\276\265a\240\276\230\375\251=\236\205\205>\231\016\016?\274j\315;\"U\\\276~\321\236>H\234.\276[z\021\277W\352,\276\211k\021\2777\217\271=N\036\233\276M\354\254\275sw\317<\211\2713\277\t;\037\277Y\212,\277\363\036Y>\263\020\231\276)\031+\277\332\3306\276\263\251\033\277J\177\302\275\000y\222\275E\361\000<\221\231\354\275\357H\232<\307q\245\275\316\033\003=?\264\013\276,DK=\316\267\264>\232\244(\274\236\032G=\246w\272\275\306\t\001\275B\316%\276\273 \212=\31

In [44]:
import onnx
from onnx import helper
from onnx import TensorProto
import numpy as np
import onnxruntime as rt

def create_single_lstm_onnx_model(
    model_name: str = "single_lstm_model.onnx",
    input_size: int = 10,
    hidden_size: int = 64,
    batch_size: int = 1,
    sequence_length: int = 5,
    direction: str = "forward", # "forward", "reverse", or "bidirectional"
    activations: list = None, # e.g., ["Sigmoid", "Tanh", "Tanh"]
    bias: bool = True
):
    """
    Manually creates an ONNX model with a single LSTM layer.

    Args:
        model_name (str): The name of the ONNX file to save.
        input_size (int): The number of expected features in the input `X`.
        hidden_size (int): The number of features in the hidden state `H`.
        batch_size (int): The batch size for the input.
        sequence_length (int): The length of the input sequence.
        direction (str): The direction of the LSTM. Can be "forward", "reverse", or "bidirectional".
        activations (list, optional): List of strings for activation functions.
                                      Order: [f, g, h] (forget, cell, output).
                                      Defaults to ["Sigmoid", "Tanh", "Tanh"] if None.
        bias (bool): Whether the LSTM layer includes bias terms.
    """
    print(f"--- Creating ONNX LSTM Model: {model_name} ---")
    print(f"Input Size: {input_size}, Hidden Size: {hidden_size}")
    print(f"Batch Size: {batch_size}, Sequence Length: {sequence_length}")
    print(f"Direction: {direction}, Activations: {activations}, Bias: {bias}")

    # Determine number of directions for weight/bias shapes
    num_directions = 0
    if direction == "forward" or direction == "reverse":
        num_directions = 1
    elif direction == "bidirectional":
        num_directions = 2
    else:
        raise ValueError("Direction must be 'forward', 'reverse', or 'bidirectional'")

    # Default activations if not provided
    if activations is None:
        activations = ["Sigmoid", "Tanh", "Tanh"] # f, g, h

    # 1. Define Model Inputs
    # X: input sequence (seq_len, batch_size, input_size)
    X = helper.make_tensor_value_info('X', TensorProto.FLOAT, [sequence_length, batch_size, input_size])
    # initial_h: initial hidden state (num_directions, batch_size, hidden_size)
    initial_h = helper.make_tensor_value_info('initial_h', TensorProto.FLOAT, [num_directions, batch_size, hidden_size])
    # initial_c: initial cell state (num_directions, batch_size, hidden_size)
    initial_c = helper.make_tensor_value_info('initial_c', TensorProto.FLOAT, [num_directions, batch_size, hidden_size])

    # 2. Define Initializers (Weights and Biases)
    # W: Weight tensor (num_directions, 4*hidden_size, input_size)
    # (W_ii|W_if|W_ic|W_io) for each direction
    W_shape = [num_directions, 4 * hidden_size, input_size]
    W = helper.make_tensor('W', TensorProto.FLOAT, W_shape,
                           np.random.rand(*W_shape).astype(np.float32).flatten())

    # R: Recurrence weight tensor (num_directions, 4*hidden_size, hidden_size)
    # (R_hi|R_hf|R_hc|R_ho) for each direction
    R_shape = [num_directions, 4 * hidden_size, hidden_size]
    R = helper.make_tensor('R', TensorProto.FLOAT, R_shape,
                           np.random.rand(*R_shape).astype(np.float32).flatten())

    # B: Bias tensor (num_directions, 8*hidden_size)
    # (W_ib|W_fb|W_cb|W_ob|R_ib|R_fb|R_cb|R_ob) for each direction
    # If bias=False, this initializer should not be present.
    # initializers = [W_ori, R_ori]
    initializers = [W, R]
    lstm_node_inputs = ['X', 'W', 'R'] # Required inputs for LSTM node

    if bias:
        # B_shape = [num_directions, 8 * hidden_size]
        # B = helper.make_tensor('B', TensorProto.FLOAT, B_shape,
        #                        np.random.rand(*B_shape).astype(np.float32).flatten())
        initializers.append(B_ori)
        # initializers.append(B)
        lstm_node_inputs.append('B')
    else:
        # If no bias, B input is optional and should be an empty string
        lstm_node_inputs.append('') # Placeholder for optional B input

    # Add initial_h and initial_c to node inputs
    lstm_node_inputs.extend(['', 'initial_h', 'initial_c']) # sequence_lens is empty string

    # 3. Define Model Outputs
    # Y: Output sequence (seq_len, num_directions, batch_size, hidden_size)
    # Note: The ONNX LSTM output Y has num_directions as its second dimension.
    Y = helper.make_tensor_value_info('Y', TensorProto.FLOAT, [sequence_length, num_directions, batch_size, hidden_size])
    # Y_h: Final hidden state (num_directions, batch_size, hidden_size)
    Y_h = helper.make_tensor_value_info('Y_h', TensorProto.FLOAT, [num_directions, batch_size, hidden_size])
    # Y_c: Final cell state (num_directions, batch_size, hidden_size)
    Y_c = helper.make_tensor_value_info('Y_c', TensorProto.FLOAT, [num_directions, batch_size, hidden_size])

    # 4. Create the LSTM Node
    lstm_node_attributes = {
        "hidden_size": hidden_size,
        "direction": direction,
        "activations": activations
    }

    lstm_node = helper.make_node(
        'LSTM',
        inputs=lstm_node_inputs,
        outputs=['Y', 'Y_h', 'Y_c'],
        name='my_lstm_node',
        **lstm_node_attributes
    )

    # 5. Create the Graph
    graph_def = helper.make_graph(
        [lstm_node],
        'lstm_graph',
        [X, initial_h, initial_c], # Graph inputs
        [Y, Y_h, Y_c],             # Graph outputs
        initializer=initializers  # Initializers (weights, biases)
    )

    # 6. Create the Model
    # Set opset version (e.g., 14 for LSTM)
    opset_imports = [helper.make_opsetid("ai.onnx", 10)] # LSTM is in ai.onnx domain
    model_def = helper.make_model(graph_def, producer_name='manual-onnx-lstm', opset_imports=opset_imports, ir_version=10)

    # 7. Save the Model
    onnx.save(model_def, model_name)
    print(f"ONNX LSTM model saved to {model_name}")

    return model_name

def verify_onnx_model(model_path: str, input_size: int, hidden_size: int,
                       batch_size: int, sequence_length: int, num_directions: int):
    """
    Loads the created ONNX model and runs a dummy inference to verify its structure.
    """
    print(f"\n--- Verifying ONNX Model: {model_path} ---")
    try:
        session = rt.InferenceSession(model_path, providers=['CPUExecutionProvider'])
        print("ONNX Runtime session created successfully.")

        # Prepare dummy input data
        dummy_X = np.random.rand(sequence_length, batch_size, input_size).astype(np.float32)
        dummy_initial_h = np.zeros((num_directions, batch_size, hidden_size), dtype=np.float32)
        dummy_initial_c = np.zeros((num_directions, batch_size, hidden_size), dtype=np.float32)

        input_names = [input.name for input in session.get_inputs()]
        output_names = [output.name for output in session.get_outputs()]

        print(f"Model Inputs: {input_names}")
        print(f"Model Outputs: {output_names}")

        # Create input feed dictionary
        input_feed = {
            'X': dummy_X,
            'initial_h': dummy_initial_h,
            'initial_c': dummy_initial_c
        }

        # Run inference
        outputs = session.run(output_names, input_feed)

        print("Inference successful.")
        print(f"Output Y shape: {outputs[0].shape} (expected: ({sequence_length}, {num_directions}, {batch_size}, {hidden_size}))")
        print(f"Output Y_h shape: {outputs[1].shape} (expected: ({num_directions}, {batch_size}, {hidden_size}))")
        print(f"Output Y_c shape: {outputs[2].shape} (expected: ({num_directions}, {batch_size}, {hidden_size}))")

        # Basic shape validation
        assert outputs[0].shape == (sequence_length, num_directions, batch_size, hidden_size)
        assert outputs[1].shape == (num_directions, batch_size, hidden_size)
        assert outputs[2].shape == (num_directions, batch_size, hidden_size)
        print("Output shapes match expectations.")

    except Exception as e:
        print(f"Error verifying ONNX model: {e}")

if __name__ == "__main__":
    # Example 1: Unidirectional LSTM (forward) with default activations
    model_path_forward = create_single_lstm_onnx_model(
        model_name="lstm_forward_default_activations.onnx",
        input_size=10,
        hidden_size=64,
        batch_size=1,
        sequence_length=5,
        direction="forward",
        activations=["Sigmoid", "Tanh", "Tanh"] # Explicitly specify common defaults
    )
    verify_onnx_model(model_path_forward, 10, 64, 2, 5, 1)



--- Creating ONNX LSTM Model: lstm_forward_default_activations.onnx ---
Input Size: 10, Hidden Size: 64
Batch Size: 1, Sequence Length: 5
Direction: forward, Activations: ['Sigmoid', 'Tanh', 'Tanh'], Bias: True
ONNX LSTM model saved to lstm_forward_default_activations.onnx

--- Verifying ONNX Model: lstm_forward_default_activations.onnx ---
ONNX Runtime session created successfully.
Model Inputs: ['X', 'initial_h', 'initial_c']
Model Outputs: ['Y', 'Y_h', 'Y_c']
Error verifying ONNX model: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: initial_h for the following indices
 index: 1 Got: 2 Expected: 1
 Please fix either the inputs/outputs or the model.
